# Audiobooks business case

### Extract the data from the csv

In [10]:
import numpy as np
from sklearn import preprocessing
import numpy as np
import tensorflow as tf

raw_csv_data = np.loadtxt('Audiobooks_data.csv',delimiter=',')
unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

### Balance the dataset

In [6]:
num_one_targets = int(np.sum(targets_all))

zero_targets_counter = 0

indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

# Create two new variables, one that will contain the inputs, and one that will contain the targets.
# We delete all indices that we marked "to remove" in the loop above.
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

### Standardize the inputs

In [7]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### Shuffle the data

In [8]:
# Shuffle the indices of the data, so the data is not arranged in any way when we feed it.
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

# Use the shuffled indices to shuffle the inputs and targets.
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Split the dataset into train, validation, and test

In [9]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]


# Print the number of targets that are 1s, the total number of samples, and the proportion for training, validation, and test (~50%).
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1772.0 3579 0.4951103660240291
228.0 447 0.5100671140939598
237.0 448 0.5290178571428571


### Save/Load the three datasets in *.npz (Optional)


In [ ]:
# Save the three datasets in *.npz.
# In the next lesson, you will see that it is extremely valuable to name them in such a coherent way!

np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

In [ ]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')
# we can load the inputs and the targets in the same line
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

# we load the test data in the temporary variable
npz = np.load('Audiobooks_data_test.npz')
# we create 2 variables that will contain the test inputs and the test targets
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

## Model Outline

In [11]:
# Set the input and output sizes
input_size = 10
output_size = 2
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 50

# define how the model will look like
model = tf.keras.Sequential([
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])


### Choose the optimizer and the loss function

# we define the optimizer we'd like to use,
# the loss function,
# and the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training
# That's where we train the model we have built.

# set the batch size
batch_size = 100

# set a maximum number of training epochs
max_epochs = 100

# set an early stopping mechanism
# let's set patience=2, to be a bit tolerant against random validation loss increases
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

# fit the model
# note that this time the train, validation and test data are not iterable
model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          # callbacks are functions called by a task when a task is completed
          # task here is to check if val_loss is increasing
          callbacks=[early_stopping], # early stopping
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2 # making sure we get enough information about the training process
          )

Epoch 1/100
36/36 - 2s - loss: 0.5256 - accuracy: 0.8069 - val_loss: 0.4014 - val_accuracy: 0.8837 - 2s/epoch - 69ms/step
Epoch 2/100
36/36 - 0s - loss: 0.3474 - accuracy: 0.8793 - val_loss: 0.3373 - val_accuracy: 0.8859 - 108ms/epoch - 3ms/step
Epoch 3/100
36/36 - 0s - loss: 0.3038 - accuracy: 0.8885 - val_loss: 0.3082 - val_accuracy: 0.8904 - 112ms/epoch - 3ms/step
Epoch 4/100
36/36 - 0s - loss: 0.2840 - accuracy: 0.8919 - val_loss: 0.2913 - val_accuracy: 0.8926 - 108ms/epoch - 3ms/step
Epoch 5/100
36/36 - 0s - loss: 0.2715 - accuracy: 0.8969 - val_loss: 0.2824 - val_accuracy: 0.8949 - 101ms/epoch - 3ms/step
Epoch 6/100
36/36 - 0s - loss: 0.2632 - accuracy: 0.9005 - val_loss: 0.2756 - val_accuracy: 0.8993 - 105ms/epoch - 3ms/step
Epoch 7/100
36/36 - 0s - loss: 0.2567 - accuracy: 0.9047 - val_loss: 0.2704 - val_accuracy: 0.8971 - 110ms/epoch - 3ms/step
Epoch 8/100
36/36 - 0s - loss: 0.2534 - accuracy: 0.9025 - val_loss: 0.2731 - val_accuracy: 0.8993 - 148ms/epoch - 4ms/step
Epoch 9/10

## Test the model

In [12]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

14/14 [==============================] - 0s 2ms/step - loss: 0.2539 - accuracy: 0.9040

Test loss: 0.25. Test accuracy: 90.40%
